In [1]:
# from google.colab import drive
# drive.mount("/content/drive/")

In [2]:

import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import shutil
import sys

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel

ModuleNotFoundError: No module named 'transformers'

In [3]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cpu'

In [4]:
dataset = pd.read_csv('drive/MyDrive/NLP_Final_Project/dataset/quotes.csv', delimiter=',')
nRow, nCol = dataset.shape
print(f'There are {nRow} rows and {nCol} columns')

FileNotFoundError: [Errno 2] No such file or directory: 'drive/MyDrive/NLP_Final_Project/dataset/quotes.csv'

In [5]:

# Null Value check
print(dataset.isnull().sum())
dataset.dropna(inplace=True)
dataset.quote.drop_duplicates(inplace=True)

# Remove commas from categories. ['Life, Motivation']
dataset['category'] = dataset['category'].str.replace(', ', ' ')

# Strip extra spaces in category
dataset['category'] = dataset['category'].str.strip()
dataset['quote'] = dataset['quote'].str.strip()
dataset['author'] = dataset['author'].str.strip()

# Convert upper case to lower case
dataset['category'] = dataset['category'].str.lower()
dataset['quote'] = dataset['quote'].str.lower() ## comment if needed
dataset['author'] = dataset['author'].str.lower() ## comment if needed

NameError: name 'dataset' is not defined

In [6]:
# Creating a dictionary to store count for all tags.

tags_dict = {}

for row in dataset.itertuples():
    for cat in row.category.split(' '):
        if cat=='':
            continue
        tags_dict[cat.lower()] = tags_dict.get(cat.lower(), 0) + 1

sorted_tags_dict = dict(sorted(tags_dict.items(), key=lambda x: x[1], reverse=True))

NameError: name 'dataset' is not defined

In [7]:
# Remove all the tags whose count is below the specified threshold

threshold = 1500

tags_to_remove = [key for key, value in sorted_tags_dict.items() if value < threshold]

# Remove items from the dictionary
for key in tags_to_remove:
    del sorted_tags_dict[key]

print(f'After removing less frequent tags: there are {len(sorted_tags_dict)} types of different tags')

# Null Value check
dataset.dropna(inplace=True)

NameError: name 'sorted_tags_dict' is not defined

In [8]:
# List of tags to keep
top_k = 5
tags_to_keep = [tag for tag in list(sorted_tags_dict.keys())[:top_k]]
print(f'Top {top_k} tags are {tags_to_keep}')

# Remove all the rows where the quote lenght longer than a certain length.
max_length_threshold = 256
filtered_dataset = dataset[dataset['quote'].apply(len) <= max_length_threshold]
dataset = filtered_dataset


dataset['tags'] = dataset['category'].apply(lambda x: [1 if tag in x.split() else 0 for tag in tags_to_keep])

# Remove quotes which are not related to any of the chosen tags
dataset = dataset[dataset['tags'].apply(lambda x: not all(elem == 0 for elem in x))]

NameError: name 'sorted_tags_dict' is not defined

In [9]:
seed = 20

dataset_size = 20000
sampled_df = dataset.sample(n=dataset_size, random_state=seed).reset_index(drop=True)

tags_df = pd.DataFrame(sampled_df['tags'].to_list(), columns=tags_to_keep)

sampled_df = pd.concat([sampled_df, tags_df], axis=1)

sampled_df = sampled_df.drop(['author','category','tags'], axis=1)
# sampled_df = sampled_df.drop(['author','category'], axis=1)
sampled_df.head()

NameError: name 'dataset' is not defined

In [10]:
train_ratio = 0.7
test_ratio = 0.15
valid_ratio = 1 - train_ratio - test_ratio

# Split the data into training, validation, and test sets (80% train, 10% validation, 10% test)
train_df, temp_df = train_test_split(sampled_df, test_size=test_ratio, random_state=seed)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=seed)

train_df, test_df, valid_df = train_df.reset_index(drop=True), test_df.reset_index(drop=True), valid_df.reset_index(drop=True)

NameError: name 'sampled_df' is not defined

In [11]:
print(train_df.shape)
print(test_df.shape)
print(valid_df.shape)

NameError: name 'train_df' is not defined